## Interfacing with XADC

TODO: Update code to DMA XADC

The AD2 uses I2C protocol so we have to use the I2C communication. The PS is what handles the communication over I2C devices. Our software on the ARM has to read from the device and then what I think makes sense is we have a DMA that can transfer the large amount of data between the memory and peripheral (AD2) in the PL. Then afterward the DMA will bring our data back to PS and it will go through the downsampled code.

In [ ]:
import pynq
from pynq import GPIO

__author__ = "Adam Taylor"
__copyright__ = "Copyright 2020, Adiuvo"
__email__ = "Adam@adiuvoengineering.com"


class xadc_scopeOverlay(pynq.Overlay):
    """.
    """
    def __init__(self, bitfile, **kwargs):
        super().__init__(bitfile, **kwargs)
        if self.is_loaded():
            pass

In [ ]:
ol = Overlay('receiver_arm_bitstream/design_1.bit')
ol.download()

In [ ]:
ol?

In [ ]:
import numpy as np
from pynq import allocate
from pynq import Overlay
import matplotlib.pyplot as mp

input_buffer = allocate(shape=(128,), dtype=np.uint16)

dma = overlay.axi_dma_0
recvstatus = dma.recvchannel.running

dma.buffer_max_size
samples = 0
maxSamples = 40*128
xAdcVoltages = np.copy(input_buffer)


while samples < maxSamples:
    dma.recvchannel.transfer(input_buffer)
    await dma.recvchannel.wait_async()
    print(input_buffer)   
#     np.append(xAdcVoltages,input_buffer)
    samples += 128

    mp.plot(input_buffer)
    mp.show()

iic_data = [0x28]
rx_data = bytes(1)

addr = 0x28
iic.send(addr, iic_data, len(iic_data),1)
iic.receive(addr,rx_data,2,0)

In [ ]:
# Initialize the plot
plt.figure(figsize=(10, 6))
plt.xlabel('Time')
plt.ylabel('Received Data')
plt.title('Continuous Data Plot')

addr = 0x28 # or 0x4C for write, 0x4D for read
iic_data = [0x28]

while True:
    # Send data
    iic.send(addr, iic_data, len(iic_data), 1)
    
    # Receive data
    rx_data = bytearray(4)  # Adjusted for 32-bit data
    iic.receive(addr, rx_data, 4, 0)  # Receive 4 bytes
    
    # Get the current time
    current_time = time.time()
    
    # Convert the received data to an integer
    received_value = int.from_bytes(rx_data, byteorder='big')
    
    # Plot the latest received data point
    plt.scatter(current_time, received_value, color='b')
    plt.gcf().autofmt_xdate()
    plt.draw()
    plt.pause(0.01)
    
    # Clear the previous output
    clear_output(wait=True)
    display(plt.gcf())
    
    # Add a delay if needed
    time.sleep(0.5)  # Adjust the delay time as per your requirement

## Interfacing with DMA to send information to Receiver IP Block

In [1]:
from pynq import Overlay
import numpy as np
from pynq import allocate
from pynq.lib import dma
from scipy.linalg import dft
import matplotlib.pyplot as plt

In [2]:
ol=Overlay('receiver_arm_bitstream/design_1.bit')

In [3]:
ol?

In [4]:
dma1=ol.axi_dma_0
dma2=ol.axi_dma_1

In [5]:
NUM_SAMPLES=236
in_I = allocate(shape=(128,), dtype=np.float32) 
out_I = allocate(shape=(128,), dtype=np.float32) 
out_Q = allocate(shape=(128,), dtype=np.float32)

with open("tx_out.dat", mode='rb') as file:
    file_content = file.read()

float_strs = file_content.split(b" ")
volt_floats = [float(x) for x in float_strs if x != b'']
volt_array = np.array(volt_floats)
print(volt_array.shape)
print(volt_array[0:128])

num_chunks = volt_array.shape[0] // 128
print(num_chunks)
# for i in range(num_chunks):
#     start_index = i * 128
#     end_index = start_index + 128
#     in_I[:] = volt_array[start_index:end_index]
    
#     print(in_I.shape)
    


    

(3776,)
[ 2.84524e-01 -4.19076e-01  1.88752e-01  2.18882e-01 -4.44955e-01
  2.90846e-01  1.02627e-01 -3.91840e-01  3.40343e-01 -1.33892e-02
 -3.00801e-01  3.47074e-01 -1.06692e-01 -2.07637e-01  3.37009e-01
 -1.82860e-01 -1.22364e-01  3.24795e-01 -2.52617e-01 -3.44380e-02
  2.95180e-01 -3.07435e-01  5.80836e-02  2.42343e-01 -3.37988e-01
  1.47189e-01  1.67641e-01 -3.39478e-01  2.23885e-01  8.06373e-02
 -3.17527e-01  2.87004e-01 -1.18839e-02 -2.81243e-01  3.41319e-01
 -1.10301e-01 -2.25103e-01  3.81647e-01 -2.15270e-01 -1.48328e-01
  4.05211e-01 -3.24503e-01 -4.54401e-02  3.99394e-01 -4.25972e-01
  8.23165e-02  3.50834e-01 -4.99087e-01  2.21860e-01  2.53781e-01
 -5.15613e-01  3.40016e-01  1.21984e-01 -4.76502e-01  4.25409e-01
 -1.72500e-02 -3.99822e-01  4.75517e-01 -1.50242e-01 -2.99230e-01
  4.94365e-01 -2.71442e-01 -1.81936e-01  4.83394e-01 -3.75358e-01
 -5.11299e-02  4.38647e-01 -4.57206e-01  8.64331e-02  3.60713e-01
 -5.02821e-01  2.18538e-01  2.47898e-01 -4.98634e-01  3.25568e-01
  

In [6]:

receiver_ip = ol.receiver_0
receiver_ip.write(0x00,1)
# dma3.recvchannel.start()  
# status_rx1 = dma1.recvchannel.running
# status_rx2 = dma2.recvchannel.running
# status_tx1 = dma1.sendchannel.running
dma1.sendchannel.start()
# dma2.sendchannel.start()
dma2.recvchannel.start()
dma1.recvchannel.start()


# dma3.recvchannel.transfer(out_Q) 
for i in range(num_chunks):
    start_index = i * 128
    end_index = start_index + 128
    in_I[:] = volt_array[start_index:end_index]
    
    print(in_I.shape)
    dma1.sendchannel.transfer(in_I)
#     dma2.sendchannel.transfer(in_I)
    dma2.recvchannel.transfer(out_Q)
    dma1.recvchannel.transfer(out_I)
    
    # dma3.recvchannel.wait()
    print("waiting")
    dma1.sendchannel.wait()
    dma2.recvchannel.wait()
    dma1.recvchannel.wait()
#     dma2.sendchannel.wait()
        
        
    print(out_I)
    print(out_Q)


(128,)
waiting



KeyboardInterrupt



In [ ]:
print(out_I)
print(out_I.shape)
print(out_Q)
print(out_Q.shape)

In [ ]:
plt.plot(out_I, "r")
plt.plot(out_Q, "b")
plt.show()

## Get output of DMA to C++ downsample ARM code

In [ ]:
#call C function